# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание данных:


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

Импортируем библиотеки:

In [1]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

Импортируем файл и посмотрим на него:

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head())
print(df.info())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


Думаю стоит количество звонков и колечество сообщениий привести к цeлочисленному формату.

In [3]:
df['calls'] = df['calls'].astype('int')
df['messages'] = df['messages'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null int64
minutes     3214 non-null float64
messages    3214 non-null int64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


Так же проверим пропуски и дубликаты

In [4]:
print(df.isna().sum())
print(df.duplicated().sum())

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64
0


Хоть в описании к проекту и сказано, что предоработка не нужна, проверить стоило, что все в порядке. Изменили типы данных, проверили наличие пропусков и дубликатов. 

## Разбейте данные на выборки

Выделим признаки и целевой параметр: целевой параметр - столбец "is_ultra", все остальное - признаки.

In [5]:
# для признаков удалим 'is_ultra', а для целевых параметров оставим только его:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Теперь нам необходимо разделить полученные наборы данных на тренировочную, валидационную и тестовую выборки в соотношении 60/20/20. Используем функцию train_test_split(). Для этого сначала разделим данные на два набора - обучающий и тестово-валидационный. А тестово-валидационный затем поделим пополам для разделения на валидационный и тестовый. 

In [6]:
# сначала разделим данные на тренировочный и тестовый наборы:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)

# теперь разделим тестовый на валидационный и тестовый:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

# проверим размеры датасетов
print(len(features_train))
print(len(features_valid))
print(len(features_test))

1928
643
643


## Исследуйте модели

Обучим и исследуем модели случайного леса, логической регрессии и дерево решений. Начнем со случайного леса. Переберем циклом несколько гиперпараметров и выбеим лучший.

In [8]:
# создадим переменные лучших результатов
best_model_random_forest = None
best_result_random_forest = 0

# цикл переборов деревьев
for est in range(1, 100):
    model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_random_forest.fit(features_train, target_train)
    result = model_random_forest.score(features_valid, target_valid)
    if result > best_result_random_forest:
        best_model_random_forest = model_random_forest
        best_result_random_forest = result
        
best_result_random_forest

0.7947122861586314

0.79 на валидационной выборке. Хороший результат. Но нужно проверить остальные модели, может есть вариант лучше. Логистическая регрессия:

In [9]:
# инициилизируем модель:
model_log = LogisticRegression(random_state=12345)

# обучим модель на тренировочной выборке
model_log.fit(features_train, target_train)

# получим метрику качества модели на валидационной выборке
result_log = model_log.score(features_valid, target_valid) 

# результат
result_log

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7589424572317263

Почти 0.76 - неплохой результат, но он хуже, чем модель случайного леса.

Проверим модель дерева решений.

In [10]:
# создадим переменные для сохрарнения результатов
best_model_tree = None
best_result_tree = 0

# цикл для перебора глубины дерева:
for depth in range(1, 4):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_tree = model_tree.predict(features_valid)
    result_tree = accuracy_score(target_valid, predictions_tree)
    if result_tree > best_result_tree:
        best_model_tree = model_tree
        best_result_tree = result_tree

# результат
best_result_tree

0.7853810264385692

0.78 - средний результат среди всех моделей. Но мы будем проверять модели на тестовой выборке. Предварительный итог таков:

1. 1 место по качеству - у случайного леса, accuracy почти 0.795. Самое долгое обучение, но оно того стоит. Хотя после 100 деревьев результат улучшается не сильно, мы решили высадить аж 1000 деревьев.  
2. 2 место у дерева решений. Методом перебора глубины дерева выяснилось что после 4 "узлов" качество не увеличивается, и остается примерно 0.785
3. 3 место у логистической регрессии - accurasy - почти 0.76. Хотя она и работет быстрее всех, мы нацалены на качество. 

## Проверьте модель на тестовой выборке

Проверяем модели на тестовой выборке. Начнем с логической регрессии

In [11]:
# получим метрику качества модели на тестовой выборке
result_log = model_log.score(features_test, target_test) 

# результат
result_log

0.7402799377916018

Результат совсем не удовлетворительный... На очереди дерево решений. 

In [12]:
# предсказания модели
predictions_tree = best_model_tree.predict(features_test)

# качество модели
best_result_tree = accuracy_score(target_test, predictions_tree)

# результат
best_result_tree

0.7791601866251944

Почти 0.78 - неплохо. Но может лес будет лучше?

In [13]:
# качество модели
best_result_random = best_model_random_forest.score(features_test, target_test)

# результат
best_result_random

0.7807153965785381

Результат случайного леса и дерева решений практически равны, но все же случайный лес точнее. 
Итак, наши победители:
1. RandomForest, accuracy = 0.7807
2. DecisionTree, accuracy = 0.7792
3. LogisticRegression, accuracy = 0.7403

## (бонус) Проверьте модели на адекватность

Проверкой на декватность, то есть вменяемость, как я узнал на просторах интернет, является соотнесение результатов обученных моделей с результатами моделей, которые учитывают только простые правила. Таким методом явлется DummyClassifier. С его помощью посчитаем accuracy и сравним его с нашими показателями. 

In [15]:
#
model_dummy = DummyClassifier(strategy="most_frequent")
model_dummy.fit(features_train, target_train)
model_dummy.predict(features_test)

model_dummy.score(features_test, target_test)



0.6842923794712286

Мы все таки впереди незатейливых моделей, поэтому считаю что фокус удался. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
